# Proyect 1: DMRG and Particle in a Box

### To do:
1. Sort the eigen values correctly
2. create the basis generation funciton using itertools
3. figure out why the psi_ij matrix does not match

In [1]:
import numpy as np
import scipy.linalg as la
from itertools import combinations, combinations_with_replacement


### 1 Density matrix computation

steps:

    1. Define Hamiltonian
    2. Choose system size and find the basis
    4. Compute the matrix elements
    5. Diagonalize the matrix
    6. Write Down the ground state as a matrix
    7. Compute the density matrix via matrix multiplication

### b) L = 4

Idea: Use the operators in matrix form to make the calculations

In [2]:
# definition of the operators
s_up = np.asarray([[0,1],[0,0]])
s_down = np.asarray([[0,0],[1,0]])
s_z = np.asarray([[1/2,0],[0,-1/2]])


In [3]:
def apply_operator(current_operator, next_operator,spin_chain, position):
    # applies the specified operators to the current and next place in the chain
    # args-> current_operator, next_operator: matrices representing the action on the current and 
    #        the next lattice site
    #         spin_chain: list of lists representing a basis vecotr
    #         position: index indicating the current position of the lattice
    # returns the transoformed spin chain
    spins_f = spin_chain.copy()
    # matrix multiplication only in the correct position of the basis vector
    spins_f[position] = list(np.matmul(current_operator,spin_chain[position]))
    spins_f[position+1] = list(np.matmul(next_operator,spin_chain[position+1]))    
    
    return spins_f

def calculate_matrix_term(spin_bra, spin_ket):
    # applies the heisenberg hamiltonian to each lattice site, for the hesinberg hamiltonian
    # we have 3 terms for each site.
    # args-> spin_bra: a list of lists representing the bra basis vector,
    #        spin_ket: a list of lists representing the ket basis vector
    # returns the matrix element as a number
    
    # save the spin chains just in case we need them latter
    sz_term = []
    # ket acted upon by first ladder operator product
    first_ladder = []
    # ket acted upon by second ladder operator product
    second_ladder = []
    eigen_values = []
    for i in range(0,len(spin_ket)-1):
        #Sz operator term
        transformed_spins = apply_operator(s_z,s_z,spin_ket,i)
        sz_term.append(transformed_spins)

        # First ladder operator term 
        transformed_spins = apply_operator(s_up,s_down,spin_ket,i)
        first_ladder.append(transformed_spins)

        # second ladder operator term
        transformed_spins = apply_operator(s_down,s_up,spin_ket,i)
        second_ladder.append(transformed_spins)

        # now we take the inner product with the basis Bra
        # to represent the inner product, sum the rows and then multiply all the elements to get the eigen value
        bracket = np.multiply(spin_bra,sz_term[i])
        eigen_values.append(np.prod(bracket.sum(1)))
        
        # remember that ladder operator terms have a 1/2 in front of them
        bracket = np.multiply(spin_bra,first_ladder[i])
        eigen_values.append(0.5*np.prod(bracket.sum(1)))

        bracket = np.multiply(spin_bra,second_ladder[i])
        eigen_values.append(0.5*np.prod(bracket.sum(1)))


    return np.sum(eigen_values)



Now we have to calculate all the matrix elements for the ground state part of the Hamiltonian. The ground state
is given by all the configurations corresponding to S=0

In [4]:
# Defining the ground state basis
basis_1 = [[1,0],[1,0], [0,1], [0,1]]
basis_2 = [[1,0],[0,1], [1,0], [0,1]]
basis_3 = [[1,0],[0,1], [0,1], [1,0]]
basis_4 = [[0,1],[1,0], [1,0], [0,1]]
basis_5 = [[0,1],[1,0], [0,1], [1,0]]
basis_6 = [[0,1],[0,1], [1,0], [1,0]]

basis_list = [ basis_1, basis_2, basis_3, basis_4, basis_5, basis_6]


In [25]:
# calculate each matrix element by iterating over the basis list two times
hamiltonian_matrix = np.zeros((len(basis_list), len(basis_list)))
# columns iteration
for i in range(len(basis_list)):
    # row iteration
    for j in range(len(basis_list)):
        hamiltonian_matrix[j,i] = calculate_matrix_term(basis_list[j],basis_list[i])

hamiltonian_matrix

array([[ 0.25,  0.5 ,  0.  ,  0.  ,  0.  ,  0.  ],
       [ 0.5 , -0.75,  0.5 ,  0.5 ,  0.  ,  0.  ],
       [ 0.  ,  0.5 , -0.25,  0.  ,  0.5 ,  0.  ],
       [ 0.  ,  0.5 ,  0.  , -0.25,  0.5 ,  0.  ],
       [ 0.  ,  0.  ,  0.5 ,  0.5 , -0.75,  0.5 ],
       [ 0.  ,  0.  ,  0.  ,  0.  ,  0.5 ,  0.25]])

In [32]:
# matrix diagonalization
# .eig returns a tuple of vectors
g_eigenvalues, g_eigenvectors = la.eig(np.asmatrix(hamiltonian_matrix))

print("Eigenvalues for the groundstate block")
print(g_eigenvalues)


Eigenvalues for the groundstate block
[-1.6160254 +0.j -0.95710678+0.j  0.1160254 +0.j  0.75      +0.j
  0.45710678+0.j -0.25      +0.j]


Matrix product decomposition. We now calculate $\psi_{ij}$ for the matrix representation of the ground state.
Remember that, since ground state is always at S = 0, **the matrix elements that correspond to another value of the spin have to be set to zero**.

We have to choose how to sepparate our gorund state in two systems
## for now we do it by hand

# RESARCH PYTHON PERMUTATIONS

In [33]:
# define the two system basis
A_1 = [[1,0],[1,0]]
A_2 = [[1,0],[0,1]]
A_3 = [[0,1],[1,0]]
A_4 = [[0,1],[0,1]]

A_basis = [A_1,A_2,A_3,A_4]
B_basis = [A_1,A_2,A_3,A_4]
# matrix belonging to the system basis
system_mat = np.zeros((len(A_basis), len(B_basis)))


In [34]:
# we now calculate the ground state matrix representation
psi_ij = np.zeros((len(A_basis), len(B_basis)))
#rows
for i in range(0,len(A_basis)):
    #columns
    for j in range(0,len(B_basis)):
        is_in_gbasis = A_basis[j]+B_basis[i] in basis_list
        if is_in_gbasis == True: 
            # save index of the basis vector to find the eigen value
            eigen_index = basis_list.index(A_basis[j]+B_basis[i])
            psi_ij[i,j] = g_eigenvalues[eigen_index]

rho_reduced = np.asmatrix(psi_ij)* np.asmatrix(psi_ij).H

/Users/rasputin/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: ComplexWarning: Casting complex values to real discards the imaginary part
  # This is added back by InteractiveShellApp.init_path()


In [35]:
psi_ij

array([[ 0.        ,  0.        ,  0.        , -0.25      ],
       [ 0.        , -0.95710678,  0.75      ,  0.        ],
       [ 0.        ,  0.1160254 ,  0.45710678,  0.        ],
       [-1.6160254 ,  0.        ,  0.        ,  0.        ]])

In [23]:
rho_reduced

matrix([[2.61153811, 0.        , 0.        , 0.        ],
        [0.        , 0.27144661, 0.29231269, 0.        ],
        [0.        , 0.29231269, 0.92951528, 0.        ],
        [0.        , 0.        , 0.        , 0.5625    ]])

Calculate the reduced density matrix via conjugate transposed

# Create a basis generator function one day

In [24]:
np.shape(A_basis[0])

(2, 2)

In [13]:
B_basis[0]

[[1, 0], [1, 0]]

In [14]:
a = np.matmul(A_basis[0],B_basis[0])
print(np.shape(a))
a

(2, 2)


array([[1, 0],
       [1, 0]])